In [1]:
# !pip install open3d
# import open3d
# !pip install --user open3d
# !pip3 install open3d
# !brew install libomp

In [2]:
import open3d

In [3]:
def translate_boxes_to_open3d_instance(gt_boxes):
    """
             4-------- 6
           /|         /|
          5 -------- 3 .
          | |        | |
          . 7 -------- 1
          |/         |/
          2 -------- 0
    """
    center = gt_boxes[0:3]
    lwh = gt_boxes[3:6]
    axis_angles = np.array([0, 0, gt_boxes[6] + 1e-10])
    rot = open3d.geometry.get_rotation_matrix_from_axis_angle(axis_angles)
    box3d = open3d.geometry.OrientedBoundingBox(center, rot, lwh)

    line_set = open3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)

    # import ipdb; ipdb.set_trace(context=20)
    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    line_set.lines = open3d.utility.Vector2iVector(lines)

    return line_set, box3d


In [4]:
def draw_box(vis, gt_boxes, color=(0, 1, 0), ref_labels=None, score=None):
    for i in range(gt_boxes.shape[0]):
        line_set, box3d = translate_boxes_to_open3d_instance(gt_boxes[i])
        if ref_labels is None:
            line_set.paint_uniform_color(color)
        else:
            line_set.paint_uniform_color(box_colormap[ref_labels[i]])

        vis.add_geometry(line_set)

        # if score is not None:
        #     corners = box3d.get_box_points()
        #     vis.add_3d_label(corners[5], '%.2f' % score[i])
    return vis

In [5]:
def draw_scenes(points, gt_boxes=None, ref_boxes=None, ref_labels=None, ref_scores=None, point_colors=None, draw_origin=True):
#     if isinstance(points, torch.Tensor):
#         points = points.cpu().numpy()
#     if isinstance(gt_boxes, torch.Tensor):
#         gt_boxes = gt_boxes.cpu().numpy()
#     if isinstance(ref_boxes, torch.Tensor):
#         ref_boxes = ref_boxes.cpu().numpy()

    vis = open3d.visualization.Visualizer()
    vis.create_window()

    vis.get_render_option().point_size = 1.0
    vis.get_render_option().background_color = np.zeros(3)

    # draw origin
    if draw_origin:
        axis_pcd = open3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
        vis.add_geometry(axis_pcd)

    pts = open3d.geometry.PointCloud()
    pts.points = open3d.utility.Vector3dVector(points[:, :3])

    vis.add_geometry(pts)
    if point_colors is None:
        pts.colors = open3d.utility.Vector3dVector(np.ones((points.shape[0], 3)))
    else:
        pts.colors = open3d.utility.Vector3dVector(point_colors)

    if gt_boxes is not None:
        vis = draw_box(vis, gt_boxes, (0, 0, 1))

    if ref_boxes is not None:
        vis = draw_box(vis, ref_boxes, (0, 1, 0), ref_labels, ref_scores)

    vis.run()
    vis.destroy_window()


In [6]:
# 49.35357278024932 -4.0872265244834125 6.998611167809742 1.9872066974639893 4.442183971405029 1.77456533908844 -0.0011515508553502627 car

In [7]:
import numpy as np
points = np.load('./0000000000.npy')

In [8]:
points.shape
# -31.15018083696177 -26.262241946068908 -3.497304856777191 1.441947340965271 4.673638820648193 1.8118125200271606 4.72081934997339 car
# 29.72312709756063 0.11922884379843281 -3.4968100786209106 6.273553371429443 2.3895740509033203 2.098191261291504 6.27236088892896 truck
gt_boxes = np.array([[ 29.72312709756063, 0.11922884379843281, -3.4968100786209106, 6.273553371429443, 2.3895740509033203, 2.098191261291504, 6.27236088892896  ],
                     [-12.103439991520963, -10.503559038780503, -3.4968149662017822, 6.273553371429443, 2.3895740509033203, 2.098191261291504, 3.141592653589793 ],
                    [3.099999767260745, -3.499960520781756, -3.4968149662017822, 6.273553371429443, 2.3895740509033203, 2.098191261291504,0.0],
                     [-40.19345975124162, -3.567801983350119, -3.4968149065971375, 5.565828800201416, 2.1499669551849365, 2.045147180557251, 0.0],
                     [24.15495753711546, -10.338444122095442, -3.4968149662017822, 6.36564302444458, 2.3511743545532227, 2.431375741958618, 3.1307684954134913],
                     
                    ])
# gt_boxes = np.array([[ 0,0,0,1,1,1,0 ]])


In [9]:
gt_boxes.shape

(5, 7)

In [ ]:
draw_scenes(points, gt_boxes=gt_boxes)


In [ ]:
#  "{:010d}.npy".format(seq)

In [ ]:
# # 0000000172.txt
# for i in range(150,173):
#     print("{:010d}".format(i))